## Midterm Question-2 Easy Level - Pradeep Reddy Yenreddy

### Import packages

In [1]:
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra cluster

In [2]:
clstr = Cluster()
session = clstr.connect()

### Use session to 'talk' to cassandra

I am trying to add keyspace bdmid. Dropping the bdmid if exists already

In [4]:
session.execute("DROP KEYSPACE IF EXISTS bdmid")

Finding all the keyspaces available

In [5]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


Now creating bdmid keyspace with replication factor 3. I want the data to be replicated in three nodes. so selected replication factor as 3

In [6]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS bdmid WITH REPLICATION = {
        'class':'SimpleStrategy', 
        'replication_factor':3
    }
""")

Now checking whether our keyspace is created or not

In [7]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

bdmid
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


Create a table named Songs containing fields  SongID, Title, Duration, Genre with Primary key as SongID

In [8]:
session.execute("""
CREATE TABLE IF NOT EXISTS bdmid.Songs ( 
    SongID INT, 
    Title TEXT, 
    Duration INT, 
    Genre TEXT,
    PRIMARY KEY(SongID)
);
""")

### Loading  data from CSV to Data Frame df

I have created a csv file with name Songs. I have added data to csv file and now I am loading file data into dataframe df.

In [11]:
df = pd.read_csv('Songs.csv')
df

,SongID,Title,Duration,Genre
0,1,'Shape_of_You',300,'POP'
1,2,'Let_Me_Love_you',325,'ROCK'
2,3,'I_Wanna_Fly',180,'Classical'
3,4,'Blinding_Lights',420,'ROCK'
4,5,'Natu_Natu',480,'POP'
5,6,'Hookabar',500,'Classical'


printing the dataframe with row and index

In [12]:
for index, row in df.iterrows():
    print(f"SongID = {row.SongID}, Title ={row.Title}, Duration = {row.Duration}, Genre = {row.Genre}")

SongID = 1, Title ='Shape_of_You', Duration = 300, Genre = 'POP'
SongID = 2, Title ='Let_Me_Love_you', Duration = 325, Genre = 'ROCK'
SongID = 3, Title ='I_Wanna_Fly', Duration = 180, Genre = 'Classical'
SongID = 4, Title ='Blinding_Lights', Duration = 420, Genre = 'ROCK'
SongID = 5, Title ='Natu_Natu', Duration = 480, Genre = 'POP'
SongID = 6, Title ='Hookabar', Duration = 500, Genre = 'Classical'


## Insertion

Inserting the values to the bdmid.Songs table

In [13]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO bdmid.Songs (SongID, Title, Duration, Genre)     
        VALUES ({row.SongID}, {row.Title}, {row.Duration}, {row.Genre});
        """
    )
    session.execute(f"""
        INSERT INTO bdmid.Songs (SongID, Title, Duration, Genre)     
        VALUES ({row.SongID}, {row.Title}, {row.Duration}, {row.Genre});
        """
    )


        INSERT INTO bdmid.Songs (SongID, Title, Duration, Genre)     
        VALUES (1, 'Shape_of_You', 300, 'POP');
        

        INSERT INTO bdmid.Songs (SongID, Title, Duration, Genre)     
        VALUES (2, 'Let_Me_Love_you', 325, 'ROCK');
        

        INSERT INTO bdmid.Songs (SongID, Title, Duration, Genre)     
        VALUES (3, 'I_Wanna_Fly', 180, 'Classical');
        

        INSERT INTO bdmid.Songs (SongID, Title, Duration, Genre)     
        VALUES (4, 'Blinding_Lights', 420, 'ROCK');
        

        INSERT INTO bdmid.Songs (SongID, Title, Duration, Genre)     
        VALUES (5, 'Natu_Natu', 480, 'POP');
        

        INSERT INTO bdmid.Songs (SongID, Title, Duration, Genre)     
        VALUES (6, 'Hookabar', 500, 'Classical');
        


Querying the table by filtering the data with primary key(SongID)

In [15]:
rows = session.execute("select (SongID, Title, Duration, Genre) from bdmid.Songs where SongID = 3")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}, {row[0][3]}")

3, I_Wanna_Fly, 180, Classical


## Update Query

Writing update query to update the song title of the song with SongID 5

In [16]:
session.execute(f"""
        Update bdmid.Songs set Title='Hello_World' where     
        SongID= 5;
        """
    )

Printing  the song with SongID 5 to see the title is updated

In [18]:
rows = session.execute("select (SongID, Title, Duration, Genre) from bdmid.Songs where SongID = 5")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}, {row[0][3]}")

5, Hello_World, 480, POP


## Delete query

Deleting song with SongID=6

In [19]:
session.execute(f"""
        Delete from bdmid.Songs where     
        SongID=6;
        """
    )

Printing the values of the table. We could see that SongID 6 is removed from the table

In [21]:
rows = session.execute("select (SongID, Title, Duration, Genre) from bdmid.Songs")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}, {row[0][3]}")

5, Hello_World, 480, POP
1, Shape_of_You, 300, POP
2, Let_Me_Love_you, 325, ROCK
4, Blinding_Lights, 420, ROCK
3, I_Wanna_Fly, 180, Classical
